In [ ]:
import pandas as pd
import numpy as np

from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Flatten, concatenate
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, History

In [ ]:
championdf = pd.read_csv('data/raw/champions.csv')
traindf = pd.read_csv('data/filtered/trainMatch.csv')
validdf = pd.read_csv('data/filtered/validationMatch.csv')
testdf = pd.read_csv('data/filtered/testMatch.csv')
laneAvgdf = pd.read_csv('data/filtered/laneAvg.csv')
winRatedf = pd.read_csv('data/filtered/positionalChampionWinRate.csv')
synergydf = pd.read_csv('data/filtered/synergy.csv')

newtraindf = pd.read_csv('data/filtered/newTrain.csv')
newtestdf = pd.read_csv('data/filtered/newTest.csv')

In [ ]:
def getDataAndLabel(datadf):
    categoricals = []
    stats = []
    labels = []

    for idx, row in datadf.iterrows():
        categorical = []
        
        for i in range(10):
            index = championdf.index[championdf['id'] == row[i]][0]
            
            categorical.append([0] * len(championdf))
            categorical[i][index] = 1
        categoricals.append(categorical)
        
        stat = [[],[],[],[],[],[],[],[],[],[]]

        for position in range(5):
            p1 = row[position]
            p2 = row[position+5]

            df = laneAvgdf[(laneAvgdf['position'] == position)
                           & (laneAvgdf['player 1'] == p1)
                           & (laneAvgdf['player 2'] == p2)]

            golddiff = df['gold diff'].iloc[0]
            xpdiff = df['xp diff'].iloc[0]
            windiff = df['win rate'].iloc[0]

            p1win = winRatedf[(winRatedf['position'] == position)
                              & (winRatedf['champion id'] == p1)]['win rate'].iloc[0]
            p2win = winRatedf[(winRatedf['position'] == position)
                              & (winRatedf['champion id'] == p2)]['win rate'].iloc[0]

            stat[position] += [golddiff, xpdiff, windiff, p1win]
            stat[position+5] += [-golddiff, -xpdiff, round(1-windiff, 3), p2win]

        for i in range(2):
            team = [row[i*5], row[i*5+1], row[i*5+2], row[i*5+3], row[i*5+4]]
            for position in range(5):
                for j in range(5):
                    if position != j:
                        win = synergydf[(synergydf['champion 1'] == row[i*5+position])
                                        & (synergydf['champion 2'] == row[i*5+j])]['win rate'].iloc[0]
                        stat[i*5+position].append(win)
        
        ## samples.append(list(np.array(stats).flatten()))
        stats.append(stat)
        
        if row[10] == 'blue':
            labels.append([1, 0])
        else:
            labels.append([0, 1])
    
    return np.array(categoricals), np.array(stats), np.array(labels)


In [ ]:
trainC, trainS, trainLabel = getDataAndLabel(newtraindf)
validC, validS, validLabel = getDataAndLabel(newtestdf[:200])
testC, testS, testLabel = getDataAndLabel(newtestdf[200:])

In [ ]:
trainC = np.array(trainC)
trainS = np.array(trainS)
trainLabel = np.array(trainLabel)

validC = np.array(validC)
validS = np.array(validS)
validLabel = np.array(validLabel)

testC = np.array(testC)
testS = np.array(testS)
testLabel = np.array(testLabel)

In [ ]:
es_callback = EarlyStopping(monitor='val_loss', patience=3)
history = History()

In [ ]:
def build_model_4(nodes, penalty):
    inputC = Input(shape=(10, 157))
    x = BatchNormalization()(inputC)
    x = Dense(nodes, activation='relu', kernel_regularizer=l2(penalty))(x)
    x = Dropout(0.2)(x)
    x = Dense(nodes, activation='relu', kernel_regularizer=l2(penalty))(x)
    x = BatchNormalization()(x)
    x = Model(inputs=inputC, outputs=x)

    inputS = Input(shape=(10, 8))
    y = BatchNormalization()(inputS)
    y = Dense(nodes, activation='relu', kernel_regularizer=l2(penalty))(y)
    y = Dropout(0.2)(y)
    y = Dense(nodes, activation='relu', kernel_regularizer=l2(penalty))(y)
    y = BatchNormalization()(y)
    y = Model(inputs=inputS, outputs=y)

    combined = concatenate([x.output, y.output])
    
    z = Dense(2, activation='relu')(combined)
    z = BatchNormalization()(z)
    z = Dense(nodes, activation='relu', kernel_regularizer=l2(penalty))(z)
    z = BatchNormalization()(z)
    z = Flatten()(z)
    z = Dense(2, activation='softmax')(z)
    
    model = Model(inputs=[x.input, y.input], outputs=z)
    
    model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

    return model

results = []
def test(num, nodes, penalty): 
    name = 'model_test4_' + str(num)
    model_4 = build_model_4(nodes, penalty)
    model_4.fit([trainC, trainS], trainLabel, batch_size=20, epochs=40, verbose=0,
                validation_data=([validC, validS], validLabel), callbacks=[es_callback, TensorBoard(log_dir="logs/"+name+"/")])
    result = model_4.evaluate([testC, testS], testLabel)
    results.append(result)

test(1, 50, 0.01)
test(2, 50, 0.005)
test(3, 50, 0.02)
test(4, 200, 0.005)
test(5, 200, 0.01)
test(6, 200, 0.02)

In [ ]:
build 4
1_ [0.8012090921401978, 0.5375000238418579] 50 0.01 (es) 
2_ [0.8768250942230225, 0.5412499904632568] 50 0.005 (es) 
3_ [0.8515576124191284, 0.53125]            50 0.02 (es) 
4_ [0.8115976452827454, 0.53125]           200 0.005 (es)
5_ [0.7864854335784912, 0.543749988079071] 200 0.01 (es)
6_ [0.7500364184379578, 0.556249976158142] 200 0.02 (es)
